#### Importing Libraries

In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image

#### Hyperparameters

In [2]:
batch_size = 32
classes = 6
learning_rate = 0.001
epochs = 64
# dropout_rate = 0.1

#### Initializing CUDA

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#### Preprocessing 

In [4]:
# Computed Values from values.py
# Mean: [0.6502255201339722, 0.626167893409729, 0.5942673683166504]
# Std: [0.26641741394996643, 0.263718843460083, 0.277700275182724]

In [5]:
test_transforms = transforms.Compose([transforms.Resize((224,224)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.6502, 0.6261, 0.5942],
                                                          std=[0.2664, 0.2637, 0.2777])
                                         ])

train_transforms = transforms.Compose([transforms.Resize((32, 32)), 
                                        transforms.RandomRotation(20), 
                                        transforms.RandomResizedCrop(224, scale=(0.8, 1.2)), 
                                        transforms.RandomHorizontalFlip(), 
                                        transforms.RandomAffine(degrees=0, shear=20), 
                                        transforms.RandomApply([transforms.ColorJitter(brightness=0.2, contrast=0.2)], p=0.5), 
                                        transforms.ToTensor(), 
                                        transforms.Normalize(mean=[0.6502, 0.6261, 0.5942],
                                                          std=[0.2664, 0.2637, 0.2777]) 
                                        ])

#### Importing Dataset

In [6]:
train_dataset = ImageFolder(root="./dataset/training", transform=train_transforms)
test_dataset = ImageFolder(root="./dataset/testing", transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=0)

print(f"Training dataset size: {len(train_dataset)} images")
print(f"Testing dataset size: {len(test_dataset)} images")

Training dataset size: 5568 images
Testing dataset size: 1835 images


#### Defining neural network

In [7]:
# class CNN(nn.Module):
#     def __init__(self, num_classes=5, img_height=224, img_width=224, dropout_rate=0.5):
#         super(CNN, self).__init__()
        

#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
#         self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
#         self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)

#         # self.dropout_conv = nn.Dropout(p=dropout_rate / 2) 
#         # self.dropout_fc = nn.Dropout(p=dropout_rate) 

#         final_size = img_height // 8  
#         final_size = img_width // 8

#         # Fully Connected Layers
#         self.fc1 = nn.Linear(128 * final_size * final_size, 512)
#         self.fc2 = nn.Linear(512, num_classes)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         # x = self.dropout_conv(x)  
        
#         x = self.pool(F.relu(self.conv2(x)))
#         # x = self.dropout_conv(x)
        
#         x = self.pool(F.relu(self.conv3(x)))
#         # x = self.dropout_conv(x)

#         x = x.view(x.size(0), -1)
#         x = F.relu(self.fc1(x))
#         # x = self.dropout_fc(x)  

#         x = self.fc2(x)  
#         return x

In [8]:
# class CNN(nn.Module):
#     def __init__(self, num_classes=10,img_height=32, img_width=32):
#         super(CNN, self).__init__()

#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
#         self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)

#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  

#         self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
#         self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)

#         self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) 

#         self.fc1 = nn.Linear(256 * 8 * 8, 512)  
#         self.fc2 = nn.Linear(512, num_classes) 

#     def forward(self, x):
#         x = F.relu(self.conv1(x))
#         x = F.relu(self.conv2(x))
#         x = self.pool(x) 

#         x = F.relu(self.conv3(x))
#         x = F.relu(self.conv4(x))
#         x = self.pool2(x) 

#         print("Before flattening:", x.shape)
#         x = x.view(x.size(0), -1)  
#         print("After flattening:", x.shape) 

#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x

In [9]:
class CNN(nn.Module):
    def __init__(self, num_classes=5, img_height=224, img_width=224):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        final_size = img_height // 16  
        final_size = img_width // 16
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(256 * final_size * final_size, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))

        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  
        return x

#### Setting up the model

In [10]:
# Init Model
model = CNN(classes)
# Init Loss Function
criterion = nn.CrossEntropyLoss()
# Init Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# Init Scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2,verbose=True)
total_step = len(train_loader)

c:\Users\Tushar\miniconda3\envs\torch-cuda\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


#### Fitting the Model

In [11]:
torch.cuda.empty_cache()
model.to(device)

for epoch in range(epochs):
    epoch_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):  
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_loader) 
    scheduler.step(avg_loss)  

    print('Epoch [{}/{}], Loss: {:.4f}, Learning Rate: {:.6f}'.format(epoch+1, epochs, loss.item(),optimizer.param_groups[0]["lr"]))

Epoch [1/64], Loss: 1.2724, Learning Rate: 0.001000
Epoch [2/64], Loss: 1.2065, Learning Rate: 0.001000
Epoch [3/64], Loss: 1.2297, Learning Rate: 0.001000
Epoch [4/64], Loss: 0.5002, Learning Rate: 0.001000
Epoch [5/64], Loss: 1.1028, Learning Rate: 0.001000
Epoch [6/64], Loss: 0.9160, Learning Rate: 0.001000
Epoch [7/64], Loss: 0.8945, Learning Rate: 0.001000
Epoch [8/64], Loss: 1.0467, Learning Rate: 0.001000
Epoch [9/64], Loss: 1.0594, Learning Rate: 0.001000
Epoch [10/64], Loss: 0.6875, Learning Rate: 0.001000
Epoch [11/64], Loss: 1.9038, Learning Rate: 0.001000
Epoch [12/64], Loss: 0.5753, Learning Rate: 0.001000
Epoch [13/64], Loss: 0.2671, Learning Rate: 0.001000
Epoch [14/64], Loss: 0.4198, Learning Rate: 0.001000
Epoch [15/64], Loss: 0.3325, Learning Rate: 0.001000
Epoch [16/64], Loss: 0.7312, Learning Rate: 0.001000
Epoch [17/64], Loss: 0.4503, Learning Rate: 0.001000
Epoch [18/64], Loss: 0.2492, Learning Rate: 0.001000
Epoch [19/64], Loss: 0.7805, Learning Rate: 0.001000
Ep

#### Testing

In [14]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} test images: {:.3f}%'.format(4171, 100 * correct / total))

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} train images: {:.3f}%'.format(13606, 100 * correct / total))

Accuracy of the network on the 4171 test images: 70.300%
Accuracy of the network on the 13606 train images: 95.708%


In [13]:
torch.save(model,"ecovis alpha-v0.3.pkl")